# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.11.4.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [17]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [18]:
# Distance metric functions
import numpy as np

def euclidean_distance(vec1, vec2):
    """Calculate Euclidean distance between two vectors"""
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

def manhattan_distance(vec1, vec2):
    """Calculate Manhattan distance between two vectors"""
    return np.sum(np.abs(vec1 - vec2))

def dot_product_similarity(vec1, vec2):
    """Calculate dot product similarity"""
    return np.dot(vec1, vec2)

def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors"""
    dot_product = np.dot(vec1, vec2)
    norm_a = np.linalg.norm(vec1)
    norm_b = np.linalg.norm(vec2)
    return dot_product / (norm_a * norm_b)

In [4]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using Isaac Asimov's Foundation trilogy as our sample data. .... This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [5]:
text_loader = TextFileLoader("data/Asimovfoundation.txt")
documents = text_loader.load_documents()
len(documents)

1

In [6]:
print(documents[0][:100])

THE FOUNDATION TRILOGY 
ISAAC ASIMOV 


Contents 

Introduction 

Foundation 

Foundation and Empire


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [7]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

1599

Let's take a look at some of the documents we've managed to split.

In [8]:
split_documents[0:1]

['THE FOUNDATION TRILOGY \nISAAC ASIMOV \n\n\nContents \n\nIntroduction \n\nFoundation \n\nFoundation and Empire \n\nSecond Foundation \n\nAbout the author \n\n\nTHE STORY BEHIND THE "FOUNDATION" \n\nBy ISAAC ASIMOV \n\n\nThe date was August 1, 1941. World War II had been raging for two years. France had fallen, the Battle \nof Britain had been fought, and the Soviet Union had just been invaded by Nazi Germany. The bombing \nof Pearl Harbor was four months in the future. \n\nBut on that day, with Europe in flames, and the evil shadow of Adolf Hitler apparently falling over all \nthe world, what was chiefly on my mind was a meeting toward which I was hastening. \n\nI was 2 1 years old, a graduate student in chemistry at Columbia University, and I had been writing \nscience fiction professionally for three years. In that time, I had sold five stories to John Campbell, editor \nof Astounding, and the fifth story, "Nightfall," was about to appear in the September 1941 issue of the \nmagazi

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [9]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
Each embedding model has a fixed dimension size. The dimension for text-embedding-3-small is 1536. This value cannot be changed.
2. What technique does OpenAI use to achieve this?
Embeddings are generated by feeding text through a neural network trained to map similar text inputs to similar vector representations in high-dimensional space as per the embeddings use-cases guide from OpenAI
> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #2!

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [11]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?
Async allows embedding many chunks in parallel, massively speeding up processing. It’s the fastest, most efficient way to handle large-scale embedding jobs without hitting bottlenecks.
> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [19]:
vector_db.search_by_text("What is psychohistory", k=3)


[('e was old and almost bald and he walked with a limp, but his eyes were very \nbright and blue. \n\nHe said, "I am Hari Seldon," an instant before Gaal\'s befuddled brain placed the face alongside \nthe memory of the many times he had seen it in pictures. \n\n\n4 . \n\nPSYCHOHISTORY-.. .Gaal Dornick, using nonmathematical concepts, has defined \npsychohistory to be that branch of mathematics which deals with the reactions of human \nconglomerates to fixed social and economic stimuli.... \n\n... Implicit in all these definitions is the assumption that the human conglomerate being dealt \nwith is sufficiently large for valid statistical treatment. The necessary size of such a \nconglomerate may be determined by Seldon \'s First Theorem which ...A further necessary \nassumption is that the human conglomerate be itself unaware of psychohistoric analysis in \norder that its reactions be truly random ... \n\nThe basis of all valid psychohistory lies in the development of the Seldon. Functi

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?
Set temperature=0 for deterministic behavior, and seed=42 to make randomness (if any) reproducible.
> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

### Creating and Prompting OpenAI's `gpt-4o-mini`!

Let's tie all these together and use it to prompt `gpt-4o-mini`!

In [22]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Sci-fi"),
    user_role_prompt.create_message(
        content="What is the best way to to save humanity?"
    ),
]

response = chat_openai.run(messages)

In [23]:
print(response)

The question of how to save humanity is vast and complex, often explored in the realm of science fiction, and it encompasses various dimensions, including environmental sustainability, social justice, technological advancement, and ethical considerations. Here are a few key approaches that resonate with themes often found in sci-fi narratives:

1. **Sustainability and Environmental Stewardship**: Emphasizing renewable energy sources, innovative agricultural practices, and conservation of natural resources can help mitigate climate change and preserve the planet for future generations. Sci-fi often highlights the consequences of ecological neglect, serving as a cautionary tale for our present choices.

2. **Global Cooperation**: Building bridges between nations and fostering a culture of collaboration can lead to solutions for global challenges. Sci-fi stories frequently depict how united efforts can overcome great hurdles, reminding us that working together is essential for progress.



## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [24]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [31]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [33]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Who broke the laws of robotics'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")


Response: I don't know.

Context Count: 3
Similarity Scores: ['Source 1: 0.409', 'Source 2: 0.406', 'Source 3: 0.396']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?
Incluing prompts like "Let's think this through step-by-step." or "Explain your reasoning before answering."
What is that strategy called?
chain-of-though prompting
> NOTE: You can look through ["Accessing GPT-3.5-turbo Like a Developer"](https://colab.research.google.com/drive/1mOzbgf4a2SP5qQj33ZxTz2a01-5eXqk2?usp=sharing) for an answer to this question if you get stuck!

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database

While these are suggestions, you should feel free to make whatever augmentations you desire! 

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

Made following change:
1. Imported Asimovfoundation.txt file and used this data 
2. Included eculidean, manhattan, and dot product similarity in addition to cosine

In [34]:
# Add similarity wrapper functions first
def euclidean_similarity(vec1, vec2):
    """Convert Euclidean distance to similarity score (higher = more similar)"""
    distance = np.sqrt(np.sum((vec1 - vec2) ** 2))
    return 1 / (1 + distance)

def manhattan_similarity(vec1, vec2):
    """Convert Manhattan distance to similarity score"""
    distance = np.sum(np.abs(vec1 - vec2))
    return 1 / (1 + distance)

def dot_product_similarity_wrapper(vec1, vec2):
    """Dot product similarity (already a similarity measure)"""
    return np.dot(vec1, vec2)

# Test different distance metrics
print("=== Testing Different Distance Metrics ===\n")

# Original cosine similarity
print("1. Cosine Similarity Results:")
results_cosine = vector_db.search_by_text("What is psychohistory?", k=3, distance_measure=cosine_similarity)
for i, (text, score) in enumerate(results_cosine, 1):
    print(f"   {i}. Score: {score:.4f}")
    print(f"      Text: {text[:100]}...")
    print()

# Euclidean distance (converted to similarity)
print("2. Euclidean Distance Results:")
results_euclidean = vector_db.search_by_text("What is psychohistory?", k=3, distance_measure=euclidean_similarity)
for i, (text, score) in enumerate(results_euclidean, 1):
    print(f"   {i}. Score: {score:.4f}")
    print(f"      Text: {text[:100]}...")
    print()

# Manhattan distance (converted to similarity)
print("3. Manhattan Distance Results:")
results_manhattan = vector_db.search_by_text("What is psychohistory?", k=3, distance_measure=manhattan_similarity)
for i, (text, score) in enumerate(results_manhattan, 1):
    print(f"   {i}. Score: {score:.4f}")
    print(f"      Text: {text[:100]}...")
    print()

# Dot product similarity
print("4. Dot Product Similarity Results:")
results_dot = vector_db.search_by_text("What is psychohistory?", k=3, distance_measure=dot_product_similarity_wrapper)
for i, (text, score) in enumerate(results_dot, 1):
    print(f"   {i}. Score: {score:.4f}")
    print(f"      Text: {text[:100]}...")
    print()### YOUR CODE HERE

=== Testing Different Distance Metrics ===

1. Cosine Similarity Results:
   1. Score: 0.5198
      Text: e was old and almost bald and he walked with a limp, but his eyes were very 
bright and blue. 

He s...

   2. Score: 0.4494
      Text: res no specialized knowledge. In the 
mathematics of psychohistory, of course, it works out promptly...

   3. Score: 0.4360
      Text: its full development. Psycho-history was the quintessence of sociology, it 
was the science of human...

2. Euclidean Distance Results:
   1. Score: 0.5050
      Text: e was old and almost bald and he walked with a limp, but his eyes were very 
bright and blue. 

He s...

   2. Score: 0.4880
      Text: res no specialized knowledge. In the 
mathematics of psychohistory, of course, it works out promptly...

   3. Score: 0.4849
      Text: its full development. Psycho-history was the quintessence of sociology, it 
was the science of human...

3. Manhattan Distance Results:
   1. Score: 0.0319
      Text: e was old 